### Model U-net architecture

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

In [2]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3,1,1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3,1,1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

In [3]:
class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64,128,256,512],):
        super(UNET,self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
        # Down Part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature #pass to next level
    
        # Up Part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2,feature))
    
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0],out_channels, kernel_size=1)
    
    def forward(self, x):
        skip_connections = []
    
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
    
        x = self.bottleneck(x)
    
        skip_connections = skip_connections[::-1] #reverse the connection
            
        for idx in range(0,len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if(x.shape != skip_connection.shape):
                x = TF.resize(x, size=skip_connection.shape[2:])
                
            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)
                
        return self.final_conv(x)

In [4]:
def test():
    x = torch.randn((3,1,160,160))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    print(preds.shape)
    print(x.shape)
    assert preds.shape == x.shape

In [5]:
if __name__ == "__main__":
    test()

torch.Size([3, 1, 160, 160])
torch.Size([3, 1, 160, 160])


### Dataset 

In [6]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

In [7]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class Carvana(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        # Load image and mask
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        
        # Open image and mask as PIL images
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Convert mask to grayscale

        # Convert mask to numpy array and set appropriate values (0 for background, 1 for object)
        mask = np.array(mask, dtype=np.float32)
        mask[mask == 255.0] = 1.0

        # Apply resize transformation if provided
        if self.transform:
            image = self.transform(image)
            mask = Image.fromarray(mask)
            mask = self.transform(mask)

        # Convert image to tensor
        image = transforms.ToTensor()(image)
        
        # Convert mask to tensor
        mask = np.array(mask, dtype=np.float32)
        mask = torch.tensor(mask, dtype=torch.float32)

        return image, mask

### Training

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

In [9]:
LEARNING_RATE = 1e-4
BATCH_SIZE = 4
NUM_EPOCHS = 20
shuffle=True
pin_memory=True
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 4
IMAGE_DIR = "data/train"
MASK_DIR = "data/train_masks"

In [10]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
])

In [11]:
def dice_loss(pred, target):
    smooth = 1.0
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    return 1 - (2.0 * intersection + smooth) / (union + smooth)

train_dataset = Carvana(
    image_dir=IMAGE_DIR,
    mask_dir=MASK_DIR,
    transform=transform  # Add augmentation logic if needed
)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True,
    pin_memory=True
)

In [12]:
model = UNET(in_channels=3, out_channels=1).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

In [13]:
def train():
    model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss = 0

    for batch_idx, (images, masks) in enumerate(loop):
        print(f"Batch {batch_idx + 1}/{len(train_loader)}")  # Log batch progress

        # Move data to the appropriate device
        images = images.to(DEVICE, dtype=torch.float32)
        masks = masks.to(DEVICE, dtype=torch.float32)

        # Check if images and masks are valid
        if images.size(0) == 0 or masks.size(0) == 0:
            print(f"Skipping empty batch {batch_idx + 1}")
            continue  # Skip this batch if it's empty

        # Ensure masks have the correct shape
        masks = masks.unsqueeze(1)  # Add a channel dimension if needed

        # Forward pass
        predictions = model(images)

        # Check if predictions and masks have the correct shape
        if predictions.size(1) != masks.size(1):  # Ensure they match
            print(f"Shape mismatch: predictions {predictions.size()}, masks {masks.size()}")
            continue  # Skip this batch if there's a shape mismatch

        # Calculate the loss
        loss = criterion(predictions, masks) + dice_loss(predictions, masks)

        # Add the loss to the total loss
        total_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        loop.set_postfix(loss=loss.item())
        
        # Clear batch data
        del images, masks, predictions, loss
        torch.cuda.empty_cache()

    print(f"Epoch Loss: {total_loss / len(train_loader)}")

In [ ]:
for epoch in range(NUM_EPOCHS):
    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}]")
    train()

Epoch [1/20]


  0%|                                                                                         | 0/1272 [00:00<?, ?it/s]